In [ ]:
!pip install biopython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 25.5 MB/s eta 0:00:00


In [ ]:
!unzip GCA_000005845.2_ASM584v2_genomic.zip

Archive:  GCA_000005845.2_ASM584v2_genomic.zip
  inflating: GCA_000005845.2_ASM584v2_genomic.fna  


In [ ]:
!ls -la

total 6012
drwxr-xr-x 1 root root    4096 Sep 30 15:54 .
drwxr-xr-x 1 root root    4096 Sep 30 15:52 ..
drwxr-xr-x 4 root root    4096 Sep 25 18:24 .config
-rw-r--r-- 1 root root 4699742 Sep 30 11:28 GCA_000005845.2_ASM584v2_genomic.fna
-rw-r--r-- 1 root root 1436154 Sep 30 15:54 GCA_000005845.2_ASM584v2_genomic.zip
drwxr-xr-x 1 root root    4096 Sep 25 18:24 sample_data


In [ ]:
from Bio import SeqIO
import numpy as np

# Load the genome sequence from the .fna file
fasta_file = "GCA_000005845.2_ASM584v2_genomic.fna"

def read_genome(file_path):
    """Reads the genome sequence from the given .fna file."""
    genome = ""
    for record in SeqIO.parse(file_path, "fasta"):
        genome += str(record.seq)
    return genome

def find_origin_of_replication(genome, window_size=1000, step_size=500):
    """Find regions with a high T/C ratio, which may suggest origin of replication."""
    genome_length = len(genome)
    origin_regions = []

    # Slide a window along the genome to calculate T/C ratio
    for i in range(0, genome_length - window_size, step_size):
        window_seq = genome[i:i+window_size]
        count_T = window_seq.count('T')
        count_C = window_seq.count('C')

        # Calculate T/C ratio (avoid division by zero)
        tc_ratio = count_T / (count_C + 1e-6)

        if tc_ratio > 2:  # Set a threshold for the ratio
            origin_regions.append((i, i + window_size, tc_ratio))

    # Sort regions by T/C ratio in descending order
    origin_regions.sort(key=lambda x: x[2], reverse=True)

    return origin_regions

def main():
    genome = read_genome(fasta_file)
    origin_regions = find_origin_of_replication(genome)

    print("Potential origin of replication regions (start, end, T/C ratio):")
    for region in origin_regions:
        print(f"Start: {region[0]}, End: {region[1]}, T/C Ratio: {region[2]:.2f}")

if __name__ == "__main__":
    main()

Potential origin of replication regions (start, end, T/C ratio):
Start: 583000, End: 584000, T/C Ratio: 2.97
Start: 4479500, End: 4480500, T/C Ratio: 2.76
Start: 2469000, End: 2470000, T/C Ratio: 2.69
Start: 583500, End: 584500, T/C Ratio: 2.68
Start: 3798500, End: 3799500, T/C Ratio: 2.68
Start: 1544500, End: 1545500, T/C Ratio: 2.60
Start: 1092500, End: 1093500, T/C Ratio: 2.58
Start: 2469500, End: 2470500, T/C Ratio: 2.55
Start: 1064000, End: 1065000, T/C Ratio: 2.52
Start: 4282000, End: 4283000, T/C Ratio: 2.51
Start: 527000, End: 528000, T/C Ratio: 2.45
Start: 4251000, End: 4252000, T/C Ratio: 2.45
Start: 1434000, End: 1435000, T/C Ratio: 2.44
Start: 570000, End: 571000, T/C Ratio: 2.42
Start: 4476000, End: 4477000, T/C Ratio: 2.42
Start: 331500, End: 332500, T/C Ratio: 2.42
Start: 4504500, End: 4505500, T/C Ratio: 2.40
Start: 1197500, End: 1198500, T/C Ratio: 2.38
Start: 584000, End: 585000, T/C Ratio: 2.37
Start: 1530000, End: 1531000, T/C Ratio: 2.37
Start: 1433500, End: 143450

In [ ]:
from Bio import SeqIO
from collections import Counter

def find_ori(sequence, window_size=1000, step_size=500):
    # Look for AT-rich regions
    at_content = []
    for i in range(0, len(sequence) - window_size, step_size):
        window = sequence[i:i+window_size]
        at_count = window.count('A') + window.count('T')
        at_content.append((i, at_count / window_size))

    # Find regions with highest AT content
    at_rich_regions = sorted(at_content, key=lambda x: x[1], reverse=True)[:10]

    # Look for DnaA box motifs near AT-rich regions
    dnaA_box = "TTATCCACA"  # Consensus DnaA box sequence
    potential_ori = []

    for start, _ in at_rich_regions:
        region = sequence[max(0, start-500):start+1500]
        dnaA_count = region.count(dnaA_box)
        if dnaA_count > 0:
            potential_ori.append((start, dnaA_count))

    return sorted(potential_ori, key=lambda x: x[1], reverse=True)

# Read the genome file
genome_file = "GCA_000005845.2_ASM584v2_genomic.fna"
for record in SeqIO.parse(genome_file, "fasta"):
    sequence = str(record.seq)

    print(f"Analyzing sequence: {record.id}")
    potential_ori_sites = find_ori(sequence)

    print("Potential origin of replication sites:")
    for position, dnaA_count in potential_ori_sites:
        print(f"Position: {position}, DnaA box count: {dnaA_count}")

    if not potential_ori_sites:
        print("No potential origin of replication sites found.")
    print("\n")

Analyzing sequence: U00096.3
Potential origin of replication sites:
Position: 2995000, DnaA box count: 1




In [ ]:
from Bio import SeqIO
import re

# Load the genome from the FASTA file
fasta_file = 'GCA_000005845.2_ASM584v2_genomic.fna'

# Define the DnaA box consensus motif (this may need to be adapted)
dnaa_box_motifs = [
    'TTATCCACA',  # Canonical DnaA box
    'TTATCCAC'    # Slightly shorter variant
]

# Read the genome sequence from the FASTA file
def read_genome(file):
    genome_seq = ""
    for record in SeqIO.parse(file, "fasta"):
        genome_seq += str(record.seq)
    return genome_seq

# Search for DnaA box motifs in the genome
def find_dnaa_boxes(sequence, motifs):
    motif_positions = []
    for motif in motifs:
        # Use regular expressions to find all positions of the motif
        for match in re.finditer(motif, sequence):
            motif_positions.append((match.start(), match.end(), motif))
    return motif_positions

# Main function
if __name__ == '__main__':
    genome_sequence = read_genome(fasta_file)
    dnaa_boxes = find_dnaa_boxes(genome_sequence, dnaa_box_motifs)

    # Display results
    if dnaa_boxes:
        print("DnaA box motifs found at the following positions:")
        for start, end, motif in dnaa_boxes:
            print(f"Motif: {motif}, Start: {start}, End: {end}")
    else:
        print("No DnaA box motifs found.")

DnaA box motifs found at the following positions:
Motif: TTATCCACA, Start: 984231, End: 984240
Motif: TTATCCACA, Start: 1478111, End: 1478120
Motif: TTATCCACA, Start: 1526905, End: 1526914
Motif: TTATCCACA, Start: 1544606, End: 1544615
Motif: TTATCCACA, Start: 2303999, End: 2304008
Motif: TTATCCACA, Start: 2331358, End: 2331367
Motif: TTATCCACA, Start: 2344702, End: 2344711
Motif: TTATCCACA, Start: 2969345, End: 2969354
Motif: TTATCCACA, Start: 2995467, End: 2995476
Motif: TTATCCACA, Start: 3105552, End: 3105561
Motif: TTATCCACA, Start: 3181556, End: 3181565
Motif: TTATCCACA, Start: 3204644, End: 3204653
Motif: TTATCCACA, Start: 3333932, End: 3333941
Motif: TTATCCACA, Start: 3600889, End: 3600898
Motif: TTATCCACA, Start: 3664727, End: 3664736
Motif: TTATCCACA, Start: 3883938, End: 3883947
Motif: TTATCCACA, Start: 3925980, End: 3925989
Motif: TTATCCACA, Start: 4046833, End: 4046842
Motif: TTATCCACA, Start: 4313474, End: 4313483
Motif: TTATCCACA, Start: 4392743, End: 4392752
Motif: TTATC

draw the GC skewed chart ((G-C)/(G+C)) and analyze it when falls